### Import libraries

In [2]:
import os
import obspy as obs
import numpy as np
import matplotlib.pyplot as plt

from andbro__write_stream_to_sds import __write_stream_to_sds
from andbro__querrySeismoData import __querrySeismoData


In [3]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
    bay_path = '/home/andbro/bay200/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/bay200/'
elif os.uname().nodename == 'lin-ffb-01':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/bay200/'

In [4]:
def __get_stream(seed, tbeg, tend, repository="online"):

    st = obs.Stream()

    invs = []

    print(f" -> loading {seed}...")
    try:
        st0, inv0 = __querrySeismoData(
                                    seed_id=seed,
                                    starttime=tbeg-10,
                                    endtime=tend+10,
                                    repository=repository,
                                    path=None,
                                    restitute=False,
                                    detail=None,
                                    fill_value=None,
                                    )

        st0 = st0.remove_response(inv0, output="VEL", water_level=60)

        st0 = st0.rotate('->ZNE', inventory=inv0)

        st0 = st0.trim(tbeg, tend)

        if len(st0) != 0:
            st += st0

    except Exception as e:
        print(e)
        print(f" -> failed to load data: {seed}")

    return st, invs

### Configurations

In [11]:
config = {}

config['tbeg'] = obs.UTCDateTime("2024-03-11 12:00")
config['tend'] = obs.UTCDateTime("2024-03-11 18:00")

config['tbeg'] = obs.UTCDateTime("2023-03-10 00:00")
config['tend'] = obs.UTCDateTime("2023-03-11 00:00")


config['seeds'] = [
                   # 'BW.GELB..HH*',
                   # 'BW.GRMB..HH*',
                   # 'BW.ALFT..HH*',
                   # 'BW.BIB..HH*',
                   # 'BW.TON..HH*',
                   'BW.FFB1..HH*',
                   'BW.FFB2..HH*',
                   'BW.FFB3..HH*',
                   'GR.FUR..HH*',
]

config['repository'] = "jane"

# config['path_to_out_data'] = "/home/andbro/Downloads/mseed/array/"
# config['path_to_out_data'] = data_path+"VelocityChanges/data/mseed/array/"
config['path_to_out_data'] = archive_path+f"ModalAnalysis/data/SDS/"



In [ ]:
st00 = obs.Stream()

for seed in config['seeds']:

    try:
        st0, inv0 = __get_stream(seed, config['tbeg'], config['tend'], repository=config['repository'])

        st00 += st0

        __write_stream_to_sds(st00, config['path_to_out_data'])

        del st00, st0

    except:
        print(f"-> Error: {seed}")


 -> loading BW.FFB1..HH*...


In [ ]:
# st.plot(equal_scale=False);

In [ ]:
st = st00.copy()

st = st.detrend("simple")

st = st.resample(100, no_filter=False)

st = st.detrend("simple")

st = st.filter("highpass", freq=0.01, corners=4, zerophase=True)

st = st.trim(config['tbeg'], config['tend'], nearest_sample=False)

st.plot(equal_scale=False);

In [ ]:
print(st.__str__(extended=True))

In [11]:
# force same starttime for FUR (for geopsy)
for tr in st:
    tr.stats.starttime = obs.UTCDateTime("2024-03-11T12:00:00.000000Z")

In [12]:
st

12 Trace(s) in Stream:
BW.FFB1..HHZ | 2024-03-11T12:00:00.000000Z - 2024-03-11T17:59:59.990000Z | 100.0 Hz, 2160000 samples
BW.FFB1..HHN | 2024-03-11T12:00:00.000000Z - 2024-03-11T17:59:59.990000Z | 100.0 Hz, 2160000 samples
BW.FFB1..HHE | 2024-03-11T12:00:00.000000Z - 2024-03-11T17:59:59.990000Z | 100.0 Hz, 2160000 samples
BW.FFB2..HHZ | 2024-03-11T12:00:00.000000Z - 2024-03-11T17:59:59.990000Z | 100.0 Hz, 2160000 samples
BW.FFB2..HHN | 2024-03-11T12:00:00.000000Z - 2024-03-11T17:59:59.990000Z | 100.0 Hz, 2160000 samples
BW.FFB2..HHE | 2024-03-11T12:00:00.000000Z - 2024-03-11T17:59:59.990000Z | 100.0 Hz, 2160000 samples
BW.FFB3..HHZ | 2024-03-11T12:00:00.000000Z - 2024-03-11T17:59:59.990000Z | 100.0 Hz, 2160000 samples
BW.FFB3..HHN | 2024-03-11T12:00:00.000000Z - 2024-03-11T17:59:59.990000Z | 100.0 Hz, 2160000 samples
BW.FFB3..HHE | 2024-03-11T12:00:00.000000Z - 2024-03-11T17:59:59.990000Z | 100.0 Hz, 2160000 samples
GR.FUR..HHE  | 2024-03-11T12:00:00.000000Z - 2024-03-11T17:59:59.990

## Write Data to Files

In [22]:
__write_stream_to_sds(st, config['path_to_out_data'])

creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/
creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/BW/
creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/BW/FFB1/
creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/BW/FFB1/HHZ.D
creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/BW/FFB1/HHN.D
creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/BW/FFB1/HHE.D
creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/BW/FFB2/
creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/BW/FFB2/HHZ.D
creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/BW/FFB2/HHN.D
creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/BW/FFB2/HHE.D
creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/BW/FFB3/
creating: /import/kilauea-data/VelocityChanges/data/mseed/array/2024/BW/FFB3/HHZ.D
creating: /import/kilauea-data/VelocityCha

/home/brotzer/anaconda3/envs/obs2/lib/python3.9/site-packages/obspy/io/mseed/core.py:770: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)


 -> stored stream as: 2024/BW/FFB1/HHN.D/BW.FFB1..HHN.D.2024.071
 -> stored stream as: 2024/BW/FFB1/HHE.D/BW.FFB1..HHE.D.2024.071
 -> stored stream as: 2024/BW/FFB2/HHZ.D/BW.FFB2..HHZ.D.2024.071
 -> stored stream as: 2024/BW/FFB2/HHN.D/BW.FFB2..HHN.D.2024.071
 -> stored stream as: 2024/BW/FFB2/HHE.D/BW.FFB2..HHE.D.2024.071
 -> stored stream as: 2024/BW/FFB3/HHZ.D/BW.FFB3..HHZ.D.2024.071
 -> stored stream as: 2024/BW/FFB3/HHN.D/BW.FFB3..HHN.D.2024.071
 -> stored stream as: 2024/BW/FFB3/HHE.D/BW.FFB3..HHE.D.2024.071
 -> stored stream as: 2024/GR/FUR/HHE.D/GR.FUR..HHE.D.2024.071
 -> stored stream as: 2024/GR/FUR/HHN.D/GR.FUR..HHN.D.2024.071
 -> stored stream as: 2024/GR/FUR/HHZ.D/GR.FUR..HHZ.D.2024.071
